## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,69.4541,-133.0374,44.60,53,75,16.11,broken clouds
1,1,Dikson,RU,73.5069,80.5464,35.56,77,100,19.84,overcast clouds
2,2,Thakurganj,IN,26.4500,88.1333,76.84,94,26,6.49,scattered clouds
3,3,Kodiak,US,57.7900,-152.4072,51.31,71,1,3.44,clear sky
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,83,75,4.61,broken clouds


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 28
What is the maximum temperature you would like for your trip? 94


In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,69.4541,-133.0374,44.60,53,75,16.11,broken clouds
1,1,Dikson,RU,73.5069,80.5464,35.56,77,100,19.84,overcast clouds
2,2,Thakurganj,IN,26.4500,88.1333,76.84,94,26,6.49,scattered clouds
3,3,Kodiak,US,57.7900,-152.4072,51.31,71,1,3.44,clear sky
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,83,75,4.61,broken clouds
5,5,Bambanglipuro,ID,-7.9500,110.2833,74.66,89,65,7.25,broken clouds
6,6,Ushuaia,AR,-54.8000,-68.3000,47.86,49,75,24.16,heavy intensity rain
8,8,Mataura,NZ,-46.1927,168.8643,50.49,72,58,4.72,light rain
9,9,Bluff,NZ,-46.6000,168.3333,50.81,79,82,15.19,broken clouds
10,10,Busselton,AU,-33.6500,115.3333,46.04,82,0,11.68,clear sky


In [42]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                668
City                   668
Country                664
Lat                    668
Lng                    668
Max Temp               668
Humidity               668
Cloudiness             668
Wind Speed             668
Current Description    668
dtype: int64

In [43]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                664
City                   664
Country                664
Lat                    664
Lng                    664
Max Temp               664
Humidity               664
Cloudiness             664
Wind Speed             664
Current Description    664
dtype: int64

In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuktoyaktuk,CA,44.60,broken clouds,69.4541,-133.0374,
1,Dikson,RU,35.56,overcast clouds,73.5069,80.5464,
2,Thakurganj,IN,76.84,scattered clouds,26.4500,88.1333,
3,Kodiak,US,51.31,clear sky,57.7900,-152.4072,
4,Arraial Do Cabo,BR,75.16,broken clouds,-22.9661,-42.0278,
5,Bambanglipuro,ID,74.66,broken clouds,-7.9500,110.2833,
6,Ushuaia,AR,47.86,heavy intensity rain,-54.8000,-68.3000,
8,Mataura,NZ,50.49,light rain,-46.1927,168.8643,
9,Bluff,NZ,50.81,broken clouds,-46.6000,168.3333,
10,Busselton,AU,46.04,clear sky,-33.6500,115.3333,


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"]!=""]
# clean_hotel_df.count()
clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuktoyaktuk,CA,44.60,broken clouds,69.4541,-133.0374,End of The Road Ltd.
2,Thakurganj,IN,76.84,scattered clouds,26.4500,88.1333,गुलमोहम्मद गोथरा बाबुलरेज़ा
3,Kodiak,US,51.31,clear sky,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
9,Bluff,NZ,50.81,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
11,Archena,ES,74.07,clear sky,38.1163,-1.3004,Hotel La Parra
12,Hobart,AU,47.26,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
20,Sai Buri,TH,75.99,overcast clouds,6.7013,101.6168,ร้าน​ เวดดิ้ง
30,Butaritari,KI,83.16,broken clouds,3.0707,172.7902,Isles Sunset Lodge
58,Bambous Virieux,MU,70.05,scattered clouds,-20.3428,57.7575,Casa Tia Villa
96,Vila Franca Do Campo,PT,68.41,broken clouds,37.7167,-25.4333,Pestana Bahia Praia


In [47]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure
 

Figure(layout=FigureLayout(height='420px'))